In [ ]:
import paramiko
import subprocess

ip = "172.16.16.122"
user = "root"
pw = "asd123!@"
cmd = ""
USER_HOME = "/home/team1"
HTTPD_CONF = "/etc/apache2/sites-available/000-default.conf"
APACHE_MAIN_CONF='/etc/apache2/apache2.conf'

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def set_ufw():
    cmd = """
    
    systemctl status ufw  :  현재 방화벽 확인
    systemctl stop ufw  :  방화벽 끄기
    """
    cc(cmd)

def install_package():
    cmd = """
apt update -y
apt upgrade -y
apt install -y software-properties-common inxi
apt install -y apache2
    """
    cc(cmd)

def start_apache():
    cmd = """
systemctl enable apache2
systemctl restart apache2
    """
def mk_dir():
    cc(f"mkdir -p {USER_HOME}")

def set_apa_dir():
    # 1. 홈 디렉토리 존재 확인
    cmd = f"""
if [ ! -d "{USER_HOME}" ]; then
    echo "디렉토리 {USER_HOME} 가 존재하지 않습니다."
    exit 1
fi

# 2. 권한 설정
chmod 755 {USER_HOME}

# 4. DocumentRoot 변경
sed -i 's|DocumentRoot /var/www/html|DocumentRoot {USER_HOME}|g' '{HTTPD_CONF}'

# 5. <Directory> 블록을 apache2.conf에 추가 (Ubuntu는 여기서 권한 제어함)

cp '$APACHE_MAIN_CONF" "{APACHE_MAIN_CONF}.bak_$(date +%F_%T)'

# 기존 /var/www/html 블록 수정
sed -i 's|<Directory /var/www/>|<Directory {USER_HOME}>|g' '{APACHE_MAIN_CONF}'
""" 
    cc(cmd)

def reload_apa():
    cc("systemctl reload apache2")

def main():
    set_ufw()
    install_package()
    start_apache()
    mk_dir()
    set_apa_dir()
    reload_apa()
